In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import warnings
import time
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/scraper/hillary.csv",sep=';')
df = df.drop_duplicates(subset='id')
df = df.drop(['retweets','favorites','replies','is_reply','is_retweet','url'], axis=1)
df = df.dropna()

df['user'] = df['user'].astype(str)
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
df['id'] = df['id'].astype(int)
df['text'] = df['text'].astype(str)
df['userid'] = df['userid'].astype(int)

df['loc'] = np.nan
df['state'] = np.nan
df['lat'] = np.nan
df['lon'] = np.nan
df['hashtag'] = np.nan

df_location = pd.read_csv("data/script/hillary_users.csv")
df_location = df_location[df_location['loc'].notnull()]
df_location = df_location[df_location['lat'].notnull()]

In [ ]:
def cleanText(s):
    s = s.split('http')[0]
    s = s.split('pic.twi')[0]
    h = [tag for tag in s.split() if tag.startswith("#") and len(tag) > 1]
    r = [tag for tag in s.split() if tag.startswith("@")]
    hr = h + r
    return ' '.join([t for t in s.split() if t not in hr]), h

df[['text','hashtag']] = df['text'].apply(lambda x: cleanText(x)).apply(pd.Series)

In [ ]:
df['loc'] = df['userid'].map(df_location.set_index('userid')['loc'])
df['lat'] = df['userid'].map(df_location.set_index('userid')['lat'])
df['lon'] = df['userid'].map(df_location.set_index('userid')['lon'])

df = df[df['loc'].notnull()]
df = df[df['lat'].notnull()]

print("There are %i unique tweets regarding %s that contain a location in the US" % (df.shape[0], "Hillary"))

df.head(5)

In [ ]:
import requests
import json

def getState(coordinates):
    try:
        lat, lon = coordinates
        if lat == 39.7837304 and lon == -100.4458825:
            return np.nan
        geolocator = Nominatim(country_bias="United States of America")
        location = geolocator.reverse([lat,lon])
        time.sleep(0.5)
        return location.raw['address']['state']
    except Exception as e:
        print(e)

def getStateGoogle(coordinates):
    lat, lon, loc = coordinates
    states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    
    if lat == 39.7837304 and lon == -100.4458825:
            return np.nan
    elif len(loc.split()) == 2 and loc.split()[1] in states:
        return loc.split()[1]
    
    url = "http://maps.googleapis.com/maps/api/geocode/json?"
    url += "latlng=%s,%s&sensor=false" % (lat, lon)
    v = requests.get(url).text
    j = json.loads(v)
    components = j['results'][0]['address_components']
    for c in components:
        if "administrative_area_level_1" in c['types']:
            state = c['short_name']
    return state

for i in range(2):
    s = i*300
    e = (i+1)*300
    df['state'][:1] = df[['lat','lon','loc']][:1].apply(lambda coor: getStateGoogle(coor),axis=1)
    print(i)

In [ ]:
df.to_csv("data/script/hillary_location.csv", index=False)

In [ ]:
df